<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from numpy import linalg as LA

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
#Проверим данные на наличие дубликатов
data.duplicated().sum()

153

In [6]:
#Удалим дубликаты
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


**Вывод**

Датасет с 4 признаками(пол, возраст и зарплата застрахованного, количество членов его семьи) и 
1 целевым признаком(количество страховых выплат клиенту за последние 5 лет).

Пропусков и дубликатов нет

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При преобразовании матрицы признаков $X$ матрицей $P$ формула обучения выглядит:
$$w'=P^{-1}(X^TX)^{-1}X^Ty=XP^{-1}w$$ 
При этом матрица $P$ должна быть квадратной обратимой

**Обоснование:** Если мы преобразуем матрицу признаков $X$ матрицей $P$ и получим новую матрицу $Y$, тогда $a'= Yw'$ изиенится и станет $a'= XPw'$, задача обучени изменится на $w = \arg\min_w MSE(XPw, y)$, а формула обучения как $w'=(Y^TY)^{-1}Y^Ty=((XP)^TXP)^{-1}(XP)^Ty$.

Пользуясь правилами:

$(AB)^T=A^TB^T$

$AA^{-1}=E$

$(AB)^{-1}=A^{-1}B^{-1}$ - Применимо только для $(A,B)$ квадратных, обратимых матриц.

Получим $w'=((XP)^TXP)^{-1}(XP)^Ty=(P^TX^TXP)^{-1}P^TX^Ty=P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty=P^{-1}(x^TX)^{-1}X^Ty$.


**Вывод:** При преобразовании матрицы признаков $X$ матрицей $P$ формула обучения приобретает вид: $w'=P^{-1}w$, где матрица $P$ квадратная. Из этого следует, что $a'=Yw'=XPP^{-1}w=a$, это доказывает, что предсказания должны совпадать, а качество линейной регрессии не должно измениться.

## Алгоритм преобразования

**Алгоритм**

Разделим данные на признак и целевой признак

Сгенерируем случайную обратную матрицу

Обучим модель без преобразования

Получим R2 метрику на модели без преобразования

Умножим обратную матрицу на матрицу признаков

Обученим модель на преобразованых данных

Получение R2 мерики на преобразованых данных

Сравненим эти метрики

**Обоснование**

Мы умножим признаки на квадратную невырожденную матрицу и сравним качество линейной регресии на двух матрицах: умноженной на квадратную невырожденную матрицу и не умноженой на неё.

Согласно проделанным вычислениям мы праве ожидать получения одинакового качества линейной регрессии до преобразования и после.

## Проверка алгоритма

In [7]:
features = data.drop('Страховые выплаты', axis=1).values
target = data['Страховые выплаты'].values

In [8]:
#Создадим случайный квадратный массив 4х4:
matrix_rand=np.random.normal(size = (4,4)) 
matrix_rand

array([[ 1.11895389, -1.77045269, -0.12800692,  0.52288569],
       [ 0.40651966,  0.77269022, -0.71499955, -1.38972766],
       [-0.08962593,  0.45047669, -0.72804051,  1.23323495],
       [ 0.15303442, -0.57130163, -1.25180795, -1.11274562]])

In [9]:
#Проверим обратимость марицы:
matrix_rand = LA.inv(matrix_rand)
matrix_rand

array([[ 0.6622257 ,  0.92662551,  0.14741566, -0.68271724],
       [-0.11598609,  0.54545432,  0.2580322 , -0.44975786],
       [ 0.03505413, -0.01207156, -0.48943503, -0.51088306],
       [ 0.11118924, -0.13902734,  0.43839679, -0.18692894]])

In [10]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('Метрика качества линейной регрессии R2 на исходных признаках:',r2_score(target, predictions))

matrix_new = features @ matrix_rand
model.fit(matrix_new, target)
predictions = model.predict(matrix_new)
print('Метрика качества линейной регрессии R2 на преобразованных признаках:',r2_score(target, predictions))

Метрика качества линейной регрессии R2 на исходных признаках: 0.4302010044852066
Метрика качества линейной регрессии R2 на преобразованных признаках: 0.4302010044852066


**Вывод:** Метрика R2 не сильно меняет совего значения, можно сказать, что алгоритм преобразования данных работает правильно.

**Общий вывод:** 

1. В данном проекте в качестве алгоритма шифрования предлагается умножение матрицы признаков на обратимую квадратную матрицу. Приведено теоретическое обоснование данного алгоритма.

2. Предложены два варианта его реализации. На практике подтверждено, что шифрование данных не повлияло на качество прогноза линейной регрессии. Для оценки применялась метрика качества R2.

3. Мы получили практически одинаковые предсказания на исходных и закодированных данных методом линейной регресии. Значит мы можем изменить данные до неузнаваемости при помощи умножения на случайную матрицу, которая будет являться по сути ключом к ним, и при этом получить те же предсказания.